In [1]:
import os
import pandas as pd
import glob
import re

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/08_inStrain_rug4941/")

In [3]:
gh = pd.read_csv("dbcan_out_strigent_RUG4941.csv")

In [4]:
instrain_df = glob.glob("instrain_profiles/**/*.profile_gene_info.tsv", recursive = True)

In [5]:
df_list = []

for f in instrain_df:
    df = pd.read_csv(f, sep = "\t")
    sample_id = f.split("/")[-1].split(".")[0]
    df.loc[:, "id"] = sample_id
    df_list.append(df)
    
instrain_gene_profile = pd.concat(df_list)

In [6]:
instrain_gene_profile_merged = pd.merge(instrain_gene_profile, gh, on = "gene")

In [7]:
instrain_gene_profile_filtered = instrain_gene_profile_merged.query("coverage > 5").query("breadth > 0.5")[["id", "scaffold", "gene", "CAZyme","E_value", "nucl_diversity", "dNdS_substitutions", "pNpS_variants"]].reset_index(drop = True)

In [8]:
prevelance = pd.read_csv("../R_project/microbe_species_prevelance.csv")

In [9]:
scaffold_to_genome = pd.read_csv("genomes.stb", sep = "\t", header = None)
scaffold_to_genome.columns = ["scaffold", "genome"]
scaffold_dict = {}
for f in set(instrain_gene_profile_filtered.scaffold):
    scaffold_dict[f] = list(scaffold_to_genome.query('scaffold == @f')['genome'])[0]

In [10]:
for index, row in instrain_gene_profile_filtered.iterrows():
    sf = row["scaffold"]
    genome = scaffold_dict[sf]
    instrain_gene_profile_filtered.loc[index, "genome"] = genome

In [11]:
instrain_gene_profile_filtered.genome = instrain_gene_profile_filtered.genome.apply(lambda x: x.split(".fasta")[0])

In [12]:
genome_info = pd.read_csv("genome_info.csv")[["user_genome", "Genera", "Species"]]

In [13]:
instrain_gene_profile_ready = pd.merge(instrain_gene_profile_filtered, genome_info.drop_duplicates(), left_on = "genome", right_on = "user_genome")

In [14]:
instrain_gene_profile_ready = pd.merge(instrain_gene_profile_ready, prevelance[["Species", "prevelance"]], on = "Species")

In [15]:
instrain_gene_profile_ready[["id", "scaffold", "gene", "CAZyme", "nucl_diversity", "dNdS_substitutions", "pNpS_variants", "Genera", "Species", "prevelance"]].to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/gh_microdiversity_RUG4941.csv", index = None)